In [1]:
import os
import time
import numpy as np
import pandas as pd
import tushare as ts
from tqdm import tqdm

### 爬取公司信息并保存结果

In [ ]:
TUSHARE_KEY = 'YOUR_TUSHARE_KEY'
ts.set_token(TUSHARE_KEY)
pro = ts.pro_api()
pool = pro.stock_basic(exchange='',
                        list_status='L',
                        adj='qfq',
                        fields='ts_code, symbol,name,area,industry,fullname,list_date, market,exchange,is_hs')
pool

ts_code  symbol     name area industry          fullname list_date  \
0     000001.SZ  000001     平安银行   深圳       银行        平安银行股份有限公司  19910403   
1     000002.SZ  000002      万科A   深圳     全国地产        万科企业股份有限公司  19910129   
2     000004.SZ  000004    *ST国华   深圳     软件服务    深圳国华网安科技股份有限公司  19910114   
3     000006.SZ  000006     深振业A   深圳     区域地产   深圳市振业(集团)股份有限公司  19920427   
4     000007.SZ  000007      全新好   深圳     其他商业      深圳市全新好股份有限公司  19920413   
...         ...     ...      ...  ...      ...               ...       ...   
5419  920489.BJ  920489     佳先股份   安徽     化工原料    安徽佳先功能助剂股份有限公司  20200727   
5420  920682.BJ  920682     球冠电缆   浙江     电气设备      宁波球冠电缆股份有限公司  20200727   
5421  920799.BJ  920799     艾融软件   上海     软件服务      上海艾融软件股份有限公司  20200727   
5422  920819.BJ  920819     颖泰生物   北京     农药化肥  北京颖泰嘉和生物科技股份有限公司  20200727   
5423  689009.SH  689009  九号公司-WD   北京      摩托车            九号有限公司  20201029   

     market exchange is_hs  
0        主板     SZSE     S  
1        主板     SZSE     S  
2        主板     SZSE     N  
3        主板     SZSE     S  
4        主板     SZSE     N  
...     ...      ...   ...  
5419    北交所      BSE     N  
5420    北交所      BSE     N  
5421    北交所      BSE     N  
5422    北交所      BSE     N  
5423    科创板      SSE  None  

[5424 rows x 10 columns]

In [3]:
# 因为穷没开通创业板和科创板权限，这里只考虑主板和中心板
pool_pro = pool[pool['market'].isin(['主板', '中小板'])].reset_index()
pool_pro

index    ts_code  symbol   name area industry            fullname  \
0         0  000001.SZ  000001   平安银行   深圳       银行          平安银行股份有限公司   
1         1  000002.SZ  000002    万科A   深圳     全国地产          万科企业股份有限公司   
2         2  000004.SZ  000004  *ST国华   深圳     软件服务      深圳国华网安科技股份有限公司   
3         3  000006.SZ  000006   深振业A   深圳     区域地产     深圳市振业(集团)股份有限公司   
4         4  000007.SZ  000007    全新好   深圳     其他商业        深圳市全新好股份有限公司   
...     ...        ...     ...    ...  ...      ...                 ...   
3174   4574  605580.SH  605580   恒盛能源   浙江     供气供热          恒盛能源股份有限公司   
3175   4575  605588.SH  605588   冠石科技   江苏      元器件        南京冠石科技股份有限公司   
3176   4576  605589.SH  605589   圣泉集团   山东     化工原料        济南圣泉集团股份有限公司   
3177   4577  605598.SH  605598   上海港湾   上海     建筑工程  上海港湾基础建设(集团)股份有限公司   
3178   4578  605599.SH  605599   菜百股份   北京       服饰       北京菜市口百货股份有限公司   

     list_date market exchange is_hs  
0     19910403     主板     SZSE     S  
1     19910129     主板     SZSE     S  
2     19910114     主板     SZSE     N  
3     19920427     主板     SZSE     S  
4     19920413     主板     SZSE     N  
...        ...    ...      ...   ...  
3174  20210819     主板      SSE     N  
3175  20210812     主板      SSE     N  
3176  20210810     主板      SSE     H  
3177  20210917     主板      SSE     H  
3178  20210909     主板      SSE     H  

[3179 rows x 11 columns]

In [5]:
save_path = r"../result/stock"
os.makedirs(save_path, exist_ok=True)

pool.to_csv(os.path.join(save_path, 'company_info.csv'), index=False, encoding='utf-8')
pool_pro.to_csv(os.path.join(save_path, 'company_info_pro.csv'), index=False, encoding='utf-8')

### 爬取每个公司的具体信息

In [6]:
save_path = r"../result/stock"

pool = pd.read_csv(os.path.join(save_path, 'company_info_pro.csv'), encoding='utf-8', index_col=0)
pool

ts_code  symbol   name area industry            fullname  list_date  \
index                                                                          
0      000001.SZ       1   平安银行   深圳       银行          平安银行股份有限公司   19910403   
1      000002.SZ       2    万科A   深圳     全国地产          万科企业股份有限公司   19910129   
2      000004.SZ       4  *ST国华   深圳     软件服务      深圳国华网安科技股份有限公司   19910114   
3      000006.SZ       6   深振业A   深圳     区域地产     深圳市振业(集团)股份有限公司   19920427   
4      000007.SZ       7    全新好   深圳     其他商业        深圳市全新好股份有限公司   19920413   
...          ...     ...    ...  ...      ...                 ...        ...   
4574   605580.SH  605580   恒盛能源   浙江     供气供热          恒盛能源股份有限公司   20210819   
4575   605588.SH  605588   冠石科技   江苏      元器件        南京冠石科技股份有限公司   20210812   
4576   605589.SH  605589   圣泉集团   山东     化工原料        济南圣泉集团股份有限公司   20210810   
4577   605598.SH  605598   上海港湾   上海     建筑工程  上海港湾基础建设(集团)股份有限公司   20210917   
4578   605599.SH  605599   菜百股份   北京       服饰       北京菜市口百货股份有限公司   20210909   

      market exchange is_hs  
index                        
0         主板     SZSE     S  
1         主板     SZSE     S  
2         主板     SZSE     N  
3         主板     SZSE     S  
4         主板     SZSE     N  
...      ...      ...   ...  
4574      主板      SSE     N  
4575      主板      SSE     N  
4576      主板      SSE     H  
4577      主板      SSE     H  
4578      主板      SSE     H  

[3179 rows x 10 columns]

In [10]:
stock_inf_path = r"../result/OldData"
os.makedirs(stock_inf_path, exist_ok=True)

# 设置起始日期
startdate = '20200101'
enddate = '20250811'

for j, ts_code in enumerate(pool.ts_code):
    print('正在获取第%d家，股票代码%s.' % (j+1, ts_code))
    # 接口限制访问200次/分钟，加一点微小的延时防止被ban
    path = os.path.join(stock_inf_path, ts_code + '_NormalData.csv')

    if os.path.exists(path):
        pass
    else:
        time.sleep(0.301)
        df = ts.pro_bar(ts_code=ts_code, start_date=startdate, end_date=enddate, freq='D')
        try:
            df = df.sort_values('trade_date', ascending=True)
            df.to_csv(path, index=False)
        except Exception as e:
            print(ts_code, e)

正在获取第1家，股票代码000001.SZ.
正在获取第2家，股票代码000002.SZ.
正在获取第3家，股票代码000004.SZ.
正在获取第4家，股票代码000006.SZ.
正在获取第5家，股票代码000007.SZ.
正在获取第6家，股票代码000008.SZ.
正在获取第7家，股票代码000009.SZ.
正在获取第8家，股票代码000010.SZ.
正在获取第9家，股票代码000011.SZ.
正在获取第10家，股票代码000012.SZ.
正在获取第11家，股票代码000014.SZ.
正在获取第12家，股票代码000016.SZ.
正在获取第13家，股票代码000017.SZ.
正在获取第14家，股票代码000019.SZ.
正在获取第15家，股票代码000020.SZ.
正在获取第16家，股票代码000021.SZ.
正在获取第17家，股票代码000025.SZ.
正在获取第18家，股票代码000026.SZ.
正在获取第19家，股票代码000027.SZ.
正在获取第20家，股票代码000028.SZ.
正在获取第21家，股票代码000029.SZ.
正在获取第22家，股票代码000030.SZ.
正在获取第23家，股票代码000031.SZ.
正在获取第24家，股票代码000032.SZ.
正在获取第25家，股票代码000034.SZ.
正在获取第26家，股票代码000035.SZ.
正在获取第27家，股票代码000036.SZ.
正在获取第28家，股票代码000037.SZ.
正在获取第29家，股票代码000039.SZ.
正在获取第30家，股票代码000042.SZ.
正在获取第31家，股票代码000045.SZ.
正在获取第32家，股票代码000048.SZ.
正在获取第33家，股票代码000049.SZ.
正在获取第34家，股票代码000050.SZ.
正在获取第35家，股票代码000055.SZ.
正在获取第36家，股票代码000056.SZ.
正在获取第37家，股票代码000058.SZ.
正在获取第38家，股票代码000059.SZ.
正在获取第39家，股票代码000060.SZ.
正在获取第40家，股票代码000061.SZ.
正在获取第41家，股票代码000062.SZ.
正在获取第42家，股票代码000063.SZ.
正

### 数据预处理

In [11]:
market_map = {'主板':0, '中小板':1}
exchange_map = {'SZSE':0, 'SSE':1}
is_hs_map = {'S':0, 'N':1, 'H':2}

area_map = {'深圳': 0, '北京': 1, '吉林': 2, '江苏': 3, '辽宁': 4, '广东': 5, '安徽': 6, '四川': 7, '浙江': 8,
            '湖南': 9, '河北': 10, '新疆': 11, '山东': 12, '河南': 13, '山西': 14, '江西': 15, '青海': 16, 
            '湖北': 17, '内蒙': 18, '海南': 19, '重庆': 20, '陕西': 21, '福建': 22, '广西': 23, '天津': 24, 
            '云南': 25, '贵州': 26, '甘肃': 27, '宁夏': 28, '黑龙江': 29, '上海': 30, '西藏': 31}

industry_map = {'银行': 0, '全国地产': 1, '生物制药': 2, '环境保护': 3, '区域地产': 4, '酒店餐饮': 5, '运输设备': 6, 
 '综合类': 7, '建筑工程': 8, '玻璃': 9, '家用电器': 10, '文教休闲': 11, '其他商业': 12, '元器件': 13, 
 'IT设备': 14, '其他建材': 15, '汽车服务': 16, '火力发电': 17, '医药商业': 18, '汽车配件': 19, '广告包装': 20, 
 '轻工机械': 21, '新型电力': 22, '饲料': 23, '电气设备': 24, '房产服务': 25, '石油加工': 26, '铅锌': 27, '农业综合': 28,
 '批发业': 29, '通信设备': 30, '旅游景点': 31, '港口': 32, '机场': 33, '石油贸易': 34, '空运': 35, '医疗保健': 36,
 '商贸代理': 37, '化学制药': 38, '影视音像': 39, '工程机械': 40, '软件服务': 41, '证券': 42, '化纤': 43, '水泥': 44, 
 '专用机械': 45, '供气供热': 46, '农药化肥': 47, '机床制造': 48, '多元金融': 49, '百货': 50, '中成药': 51, '路桥': 52, 
 '造纸': 53, '食品': 54, '黄金': 55, '化工原料': 56, '矿物制品': 57, '水运': 58, '日用化工': 59, '机械基件': 60, 
 '汽车整车': 61, '煤炭开采': 62, '铁路': 63, '染料涂料': 64, '白酒': 65, '林业': 66, '水务': 67, '水力发电': 68, 
 '互联网': 69, '旅游服务': 70, '纺织': 71, '铝': 72, '保险': 73, '园区开发': 74, '小金属': 75, '铜': 76, '普钢': 77, 
 '航空': 78, '特种钢': 79, '种植业': 80, '出版业': 81, '焦炭加工': 82, '啤酒': 83, '公路': 84, '超市连锁': 85, 
 '钢加工': 86, '渔业': 87, '农用机械': 88, '软饮料': 89, '化工机械': 90, '塑料': 91, '红黄酒': 92, '橡胶': 93, '家居用品': 94,
 '摩托车': 95, '电器仪表': 96, '服饰': 97, '仓储物流': 98, '纺织机械': 99, '电器连锁': 100, '装修装饰': 101, '半导体': 102, 
 '电信运营': 103, '石油开采': 104, '乳制品': 105, '商品城': 106, '公共交通': 107, '船舶': 108, '陶瓷': 109}

def JudgeST(x):
    if 'ST' in x:
        return 1
    else:
        return 0
    
col = ['open', 'high', 'low', 'pre_close',]

company_info = pd.read_csv(os.path.join(save_path, 'company_info_pro.csv'), encoding='utf-8')
company_info['is_ST'] = company_info['name'].apply(JudgeST)

# 丢弃一些多余的信息
company_info.drop(['index', 'symbol', 'fullname'], axis=1, inplace=True)
company_info.dropna(inplace=True)
company_info['market'] = company_info['market'].map(market_map)
company_info['exchange'] = company_info['exchange'].map(exchange_map)
company_info['is_hs'] = company_info['is_hs'].map(is_hs_map)

company_info

ts_code   name area industry  list_date  market  exchange  is_hs  \
0     000001.SZ   平安银行   深圳       银行   19910403       0         0      0   
1     000002.SZ    万科A   深圳     全国地产   19910129       0         0      0   
2     000004.SZ  *ST国华   深圳     软件服务   19910114       0         0      1   
3     000006.SZ   深振业A   深圳     区域地产   19920427       0         0      0   
4     000007.SZ    全新好   深圳     其他商业   19920413       0         0      1   
...         ...    ...  ...      ...        ...     ...       ...    ...   
3174  605580.SH   恒盛能源   浙江     供气供热   20210819       0         1      1   
3175  605588.SH   冠石科技   江苏      元器件   20210812       0         1      1   
3176  605589.SH   圣泉集团   山东     化工原料   20210810       0         1      2   
3177  605598.SH   上海港湾   上海     建筑工程   20210917       0         1      2   
3178  605599.SH   菜百股份   北京       服饰   20210909       0         1      2   

      is_ST  
0         0  
1         0  
2         1  
3         0  
4         0  
...     ...  
3174      0  
3175      0  
3176      0  
3177      0  
3178      0  

[3179 rows x 9 columns]

In [12]:
stock_info = pd.DataFrame()
remove_stock = []
tmp_list = []
for ts_code in tqdm(company_info['ts_code']):
    tmp_df = pd.read_csv(os.path.join(stock_inf_path, ts_code + '_NormalData.csv'))
    
    # 还需要去除一些停牌时间很久的企业
    if len(tmp_df) < 100:  # 去除一些上市不久的企业
        remove_stock.append(ts_code)
        continue
    tmp_df = tmp_df.sort_values('trade_date', ascending=True).reset_index()
    tmp_list.append(tmp_df)

stock_info = pd.concat(tmp_list)

# 定义交易日期映射
tmp_list = list(stock_info['trade_date'].unique())
date_map = dict(zip(tmp_list, range(len(tmp_list))))

ts_code_map = dict(zip(stock_info['ts_code'].unique(), range(stock_info['ts_code'].nunique())))
stock_info = stock_info.reset_index()
stock_info['ts_code_id'] = stock_info['ts_code'].map(ts_code_map)
stock_info.drop('index', axis=1, inplace=True)
stock_info['trade_date_id'] = stock_info['trade_date'].map(date_map)
stock_info['ts_date_id'] = (10000 + stock_info['ts_code_id']) * 10000 + stock_info['trade_date_id']
stock_info = stock_info.merge(company_info, how='left', on='ts_code')
stock_info_copy = stock_info.copy()

stock_info

100%|██████████| 3179/3179 [00:14<00:00, 214.94it/s]


level_0    ts_code  trade_date   open   high    low  close  \
0              0  000001.SZ    20200102  16.65  16.95  16.55  16.87   
1              1  000001.SZ    20200103  16.94  17.31  16.92  17.18   
2              2  000001.SZ    20200106  17.01  17.34  16.91  17.07   
3              3  000001.SZ    20200107  17.13  17.28  16.95  17.15   
4              4  000001.SZ    20200108  17.00  17.05  16.63  16.66   
...          ...        ...         ...    ...    ...    ...    ...   
4100114      943  605599.SH    20250805  15.48  15.61  15.20  15.27   
4100115      944  605599.SH    20250806  15.30  15.46  15.13  15.45   
4100116      945  605599.SH    20250807  15.42  16.01  15.33  15.76   
4100117      946  605599.SH    20250808  15.90  16.11  15.77  15.82   
4100118      947  605599.SH    20250811  15.68  15.76  15.43  15.50   

         pre_close  change  pct_chg  ...  trade_date_id  ts_date_id  name  \
0              NaN     NaN      NaN  ...              0   100000000  平安银行   
1            16.87    0.31     1.84  ...              1   100000001  平安银行   
2            17.18   -0.11    -0.64  ...              2   100000002  平安银行   
3            17.07    0.08     0.47  ...              3   100000003  平安银行   
4            17.15   -0.49    -2.86  ...              4   100000004  平安银行   
...            ...     ...      ...  ...            ...         ...   ...   
4100114      15.58   -0.31    -1.99  ...           1354   131601354  菜百股份   
4100115      15.27    0.18     1.18  ...           1355   131601355  菜百股份   
4100116      15.45    0.31     2.01  ...           1356   131601356  菜百股份   
4100117      15.76    0.06     0.38  ...           1357   131601357  菜百股份   
4100118      15.82   -0.32    -2.02  ...           1358   131601358  菜百股份   

         area  industry list_date market exchange  is_hs  is_ST  
0          深圳        银行  19910403      0        0      0      0  
1          深圳        银行  19910403      0        0      0      0  
2          深圳        银行  19910403      0        0      0      0  
3          深圳        银行  19910403      0        0      0      0  
4          深圳        银行  19910403      0        0      0      0  
...       ...       ...       ...    ...      ...    ...    ...  
4100114    北京        服饰  20210909      0        1      2      0  
4100115    北京        服饰  20210909      0        1      2      0  
4100116    北京        服饰  20210909      0        1      2      0  
4100117    北京        服饰  20210909      0        1      2      0  
4100118    北京        服饰  20210909      0        1      2      0  

[4100119 rows x 23 columns]

### 特征工程

In [13]:
stock_info = stock_info_copy.copy()
col = ['close', 'open', 'high', 'low']
feature_col = []
for tmp_col in col:
    stock_info[tmp_col+'_'+'transform'] = (stock_info[tmp_col] - stock_info['pre_close']) / stock_info['pre_close']
    feature_col.append(tmp_col+'_'+'transform')

In [14]:
for i in range(5):
    tmp_df = stock_info[['ts_date_id', 'close']]
    tmp_df = tmp_df.rename(columns={'close':'close_shift_{}'.format(i+1)})
    feature_col.append('close_shift_{}'.format(i+1))
    tmp_df['ts_date_id'] = tmp_df['ts_date_id'] + i + 1
    stock_info = stock_info.merge(tmp_df, how='left', on='ts_date_id')
stock_info.drop('level_0', axis=1, inplace=True)
# stock_info.dropna(inplace=True)

for i in range(5):
    stock_info['close_shift_{}'.format(i+1)] = (stock_info['close'] - stock_info['close_shift_{}'.format(i+1)]) / stock_info['close_shift_{}'.format(i+1)]

In [15]:
use_col = []
for i in range(5):
    tmp_df = stock_info[['ts_date_id', 'high', 'low']]
    tmp_df = tmp_df.rename(columns={'high':'high_shift_{}'.format(i+1), 'low':'low_shift_{}'.format(i+1)})
    use_col.append('high_shift_{}'.format(i+1))
    use_col.append('low_shift_{}'.format(i+1))
    tmp_df['ts_date_id'] = tmp_df['ts_date_id'] - i - 1
    stock_info = stock_info.merge(tmp_df, how='left', on='ts_date_id')

stock_info.dropna(inplace=True)

for i in range(5):
    stock_info['high_shift_{}'.format(i+1)] = (stock_info['high_shift_{}'.format(i+1)] - stock_info['close']) / stock_info['close']
    stock_info['low_shift_{}'.format(i+1)] = (stock_info['low_shift_{}'.format(i+1)] - stock_info['close']) / stock_info['close']

tmp_array = stock_info[use_col].values
max_increse = np.max(tmp_array, axis=1)
min_increse = np.min(tmp_array, axis=1)
stock_info['label_max'] = max_increse
stock_info['label_min'] = min_increse
stock_info['label_final'] = (stock_info['label_max'] > 0.06) & (stock_info['label_min'] > -0.03)
stock_info['label_final'] = stock_info['label_final'].apply(lambda x: int(x))
stock_info = stock_info.reset_index()
stock_info.drop('index', axis=1, inplace=True)

### 模型训练

In [16]:
trn_col = ['open', 'high', 'low', 'close', 'pre_close', 'change', 'pct_chg', 'vol', 'amount', 'ts_code_id'] + feature_col
label = 'label_final'
trn_date_min = 20200101
trn_date_max = 20241230
val_date_min = 20250101
val_date_max = 20250430
test_date_min = 20250501
test_date_max = 20250811

trn_data_idx = (stock_info['trade_date'] >= trn_date_min) & (stock_info['trade_date'] <= trn_date_max)
val_data_idx = (stock_info['trade_date'] >= val_date_min) & (stock_info['trade_date'] <= val_date_max)
test_data_idx = (stock_info['trade_date'] >= test_date_min) & (stock_info['trade_date'] <= test_date_max)

trn = stock_info[trn_data_idx][trn_col].values
trn_label = stock_info[trn_data_idx][label].values

val = stock_info[val_data_idx][trn_col].values
val_label = stock_info[val_data_idx][label].values 

test = stock_info[test_data_idx][trn_col].values
test_label = stock_info[test_data_idx][label].values

print('rate of 0: %.4f, rate of 1: %.4f' % (np.sum(trn_label==0)/len(trn_label), np.sum(trn_label==1)/len(trn_label)))
print('trn data:%d, val data:%d, test data:%d' % (len(trn), len(val), len(test)))
print('number of features:%d' % len(trn_col))

rate of 0: 0.8061, rate of 1: 0.1939
trn data:3603715, val data:244793, test data:200791
number of features:19


In [17]:
# 模型训练及评价
import lightgbm as lgb
from sklearn import metrics

params = {
        'learning_rate': 1e-3,
        'boosting_type': 'gbdt',
        'objective': 'binary',
        'metric': 'mse',
        'num_leaves':128,
        'feature_fraction': 0.8,
        'bagging_fraction': 0.8,
        'bagging_freq': 5,
        'seed': 1,
        'bagging_seed': 1,
        'lambda_l1': 0.1,
        'feature_fraction_seed': 7,
        'min_data_in_leaf': 20,
        'nthread': -1,
        'verbose': -1
    }

trn_data = lgb.Dataset(trn, trn_label)
val_data = lgb.Dataset(val, val_label)
num_round = 2000
clf = lgb.train(params, trn_data, num_round, valid_sets=[trn_data, val_data])
            
oof_lgb = clf.predict(val, num_iteration=clf.best_iteration)
test_lgb = clf.predict(test, num_iteration=clf.best_iteration)

### 模型评价

In [18]:
oof_lgb_final = np.round(oof_lgb)
print(metrics.accuracy_score(val_label, oof_lgb_final))
print(metrics.confusion_matrix(val_label, oof_lgb_final))
tp = np.sum(((oof_lgb_final == 1) & (val_label == 1)))
pp = np.sum(oof_lgb_final == 1)
print('sensitivity:%.3f'% (tp/(pp)))

0.7983561621451594
[[195387     46]
 [ 49315     45]]
sensitivity:0.495


In [19]:
thresh_hold = 0.6
oof_test_final = test_lgb >= thresh_hold
print(metrics.accuracy_score(test_label, oof_test_final))
print(metrics.confusion_matrix(test_label, oof_test_final))
tp = np.sum(((oof_test_final == 1) & (test_label == 1)))
pp = np.sum(oof_test_final == 1)
print('sensitivity:%.3f'% (tp/(pp)))

0.8144986578083679
[[163544      0]
 [ 37247      0]]
sensitivity:nan


C:\Users\slc\AppData\Local\Temp\ipykernel_34964\3488924863.py:7: RuntimeWarning: invalid value encountered in scalar divide
  print('sensitivity:%.3f'% (tp/(pp)))


In [20]:
test_postive_idx = np.argwhere(oof_test_final == 1).reshape(-1)
test_all_idx = np.argwhere(test_data_idx).reshape(-1)

# 查看选了哪些股票
tmp_col = ['ts_code', 'name', 'trade_date', 'open', 'high', 'low', 'close', 'pre_close',
       'change', 'pct_chg', 'amount', 'is_ST', 'label_max', 'label_min', 'label_final']
# stock_info.iloc[test_all_idx[test_postive_idx]]

tmp_df = stock_info[tmp_col].iloc[test_all_idx[test_postive_idx]].reset_index()
# idx_tmp = tmp_df['is_ST'] == 0
# tmp_df.loc[idx_tmp, 'is_limit_up'] = (((tmp_df['close'][idx_tmp]-tmp_df['pre_close'][idx_tmp]) / tmp_df['pre_close'][idx_tmp]) > 0.095)
# idx_tmp = tmp_df['is_ST'] == 1
# tmp_df.loc[idx_tmp, 'is_limit_up'] = (((tmp_df['close'][idx_tmp]-tmp_df['pre_close'][idx_tmp]) / tmp_df['pre_close'][idx_tmp]) > 0.047)

tmp_df['is_limit_up'] = tmp_df['close'] == tmp_df['high']

buy_df = tmp_df[(tmp_df['is_limit_up']==False)].reset_index()
buy_df.drop(['index', 'level_0'], axis=1, inplace=True)

print(len(buy_df), sum(buy_df['label_final']))
display(buy_df)

0 0


Empty DataFrame
Columns: [ts_code, name, trade_date, open, high, low, close, pre_close, change, pct_chg, amount, is_ST, label_max, label_min, label_final, is_limit_up]
Index: []